### 2D frame dataset generation for CIDRE-based retrospective flat/dark-field model estimation

Run the cells below to extract a suitable dataset of 2D images from the 3D microscopy image stacks included in *source*.
Frames exported to *outdir* will satisfy the following inclusion criteria:
* *bg_lvl* (approximate) dark background intensity
* *bg_rel_thr*  relative number of sub-threshold (*bg_lvl*) background pixels  
* *mean_rel_thr*  mean intensity relative to data type maximum

In [ ]:
import numpy as np
import random
import tifffile as tiff

from math import ceil

from pathlib import Path


##### I/O image paths

In [ ]:
# path to TIFF z-stacks
# NOTE: to be adapted...
slc = 4
source = f'/mnt/olimpo1/NIH/DANDI/000026/rawdata_biolab/I48/20210726_I48_{slc}_LeftDet_638_RightDet_561/tiff_left/rotated'
outdir = f'/mnt/NASoneScratch/michele/mic_dset/broca_638nm_I48_S{slc}'

# channel axis for RGB z-stacks (ch_ax = 1 when True, ch_ax = 3 otherwise)
ch_first = True

# objective name and fluorescence emission wavelength [nm]
obj = 'lsfm_lavision12x_dskwd'
wv = 638

# input formats
formats = ["_l.ome.tif"]


##### Export options

In [ ]:
# inclusion/exclusion criteria
bg_lvl = 150
bg_rel_thr = 0.333
mean_rel_thr = 0.005

# maximum output size [B]
max_dset_size = 100e9

# minimum z-stride between exported frames [μm]
z_strd_um = 18
px_sz_z = 3.6


##### Evaluate I/O directory content

In [ ]:
source = Path(source)
outdir = Path(outdir)


if not isinstance(wv, tuple) and not isinstance(wv, list):
    wv = (wv,)

ch_ax = 0 if ch_first else 2

# get current number of 2D frames in outdir
outdir = Path(outdir)
for c, w in enumerate(wv):
    if w != -1:
        wavedir = outdir / obj / str(w)
        if not (wavedir).is_dir():
            wavedir.mkdir(parents=True, exist_ok=True)

# get all z-stacks file paths in source
stack_lst = []
source = Path(source)  # Ensure source is a Path object
for dirpath in source.glob('*'):
    if dirpath.is_file() and any(str(dirpath).lower().endswith(fmt) for fmt in formats):
        stack_lst.append(dirpath)

random.shuffle(stack_lst)

print(f"Output directory path\n{outdir}\n\nProcessed image stacks")
for s in stack_lst:
    print(s)


##### Export 2D frames imposing the required inclusion criteria

In [ ]:
ds_size = np.zeros((len(wv),))
num_slc = np.zeros((len(wv),), dtype=int)
num_out = 1
num_stk = len(stack_lst)

z_strd = ceil(z_strd_um / px_sz_z)

for f in stack_lst:
    prc_progress = 100 * (num_out / num_stk)
    print('\nProcessing image stack {0}/{1}\t{2:0.1f}%'.format(num_out, num_stk, prc_progress), end='\r')

    # read image stack
    img = tiff.imread(f)
    img_max = np.iinfo(img.dtype).max

    # loop over z-slices
    for z in range(0, img.shape[0], z_strd):
        
        # read single frame
        slc = img[z, ...]

        # add channel axis when missing
        if slc.ndim == 2:
            slc = slc[np.newaxis, ...] if ch_first else slc[..., np.newaxis]

        # loop over channels
        for c in range(slc.shape[ch_ax]):           
            if wv[c] > 0:

                # check criteria and save "valid" z-slice to TIFF
                slc = slc[c, :, :] if ch_first else slc[..., c]
                if np.count_nonzero(slc > bg_lvl) / np.size(slc) > bg_rel_thr \
                    and np.mean(slc[slc != 0]) > mean_rel_thr * img_max:

                    tiff.imwrite(outdir / obj / str(wv[c]) / f'{num_slc[c]}.tiff', slc, compression='zlib')

                    # update total dataset size and slice counter
                    ds_size[c] += slc.itemsize * slc.size
                    num_slc[c] += 1
    
                # break if all channels reached max size
                if np.all(ds_size) >= max_dset_size:
                    break

    # increase processed stack counter
    num_out += 1

# print exported dataset information
print(f"\n\nExported data size  (objective: {obj})")
for c, w in enumerate(wv):
    if w != -1:
        print(f"- {w} nm: {num_slc[c]} images ({1e-6*np.sum(ds_size[c]):2.1f}MB)")
